### In this Notebook I focus on pipline design, the first step in making deployment of the ML more robust 

Following up on what I did in base model 0, More thorough EDA is found in this folder (Model_1). However the main focus remained on deployment and pipelines

The workflow is as follows:

### 1. using piplines

1. Processing 
    -Impute mean 
    -Impute mode and one hot incode
2. Scaling Data 
3. PCA and k best
4. RandomForestClassifier(), LogisticRegression(), SVC(), (which ever has better results)
3. Hyperparamters tunning using Gridsearch 

### 2. API Creation (Seperate .py file)



In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SelectKBest
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns  
from sklearn.metrics import f1_score
from sklearn import decomposition, datasets
from sklearn import tree
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import FeatureUnion
from sklearn.feature_selection import SelectKBest

In [6]:
df = pd.read_csv("data.csv")
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [7]:
df = df.drop(columns = 'Loan_ID')
df.head()

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [8]:
# Split the data set into training and test data 
X = df.drop(columns='Loan_Status')
y = df['Loan_Status']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=27, stratify=y)

In [9]:
# Pipline design
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SelectKBest

numeric_transform = Pipeline([('impute_mean', SimpleImputer(strategy='mean'))])

categorical_transform = Pipeline([('impute_mode', SimpleImputer(strategy='most_frequent')), 
                                  ('one-hot-encode', OneHotEncoder(sparse=False))])

Feature_Transformation = ColumnTransformer([('numeric', numeric_transform, ['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term', 'Credit_History']), 
                                        ('categorical', categorical_transform, ['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'Property_Area'])])


In [10]:
feature_union = FeatureUnion([('pca', PCA()), 
                              ('select_best', SelectKBest())])


pipe = Pipeline(steps=[('Processing',Feature_Transformation),
                       ('scaling', StandardScaler()),
                       ('features', feature_union),
                       ('classifier', 'passthrough')])

# Find the best hyperparameters and model using GridSearchCV on the train set
param_grid = {'classifier': [RandomForestClassifier(), LogisticRegression(), SVC()],    # Which is better, Logistic Regression or an SVM Classifier?
              'features__pca__n_components': [3, 5],
              'features__select_best__k': [1, 3, 6]}

# create a Grid Search object
grid = GridSearchCV(pipe, param_grid=param_grid, cv=5, scoring='accuracy')
grid.fit(X_train, y_train)


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('Processing',
                                        ColumnTransformer(transformers=[('numeric',
                                                                         Pipeline(steps=[('impute_mean',
                                                                                          SimpleImputer())]),
                                                                         ['ApplicantIncome',
                                                                          'CoapplicantIncome',
                                                                          'LoanAmount',
                                                                          'Loan_Amount_Term',
                                                                          'Credit_History']),
                                                                        ('categorical',
                                                                         Pipeline(steps=[('impute_mode',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('one-hot-encode',
                                                                                          OneH...
                                                                          'Self_Employed',
                                                                          'Property_Area'])])),
                                       ('scaling', StandardScaler()),
                                       ('features',
                                        FeatureUnion(transformer_list=[('pca',
                                                                        PCA()),
                                                                       ('select_best',
                                                                        SelectKBest())])),
                                       ('classifier', 'passthrough')]),
             param_grid={'classifier': [RandomForestClassifier(),
                                        LogisticRegression(), SVC()],
                         'features__pca__n_components': [3, 5],
                         'features__select_best__k': [1, 3, 6]},
             scoring='accuracy')

In [11]:
best_model = grid.best_estimator_
best_hyperparams = grid.best_params_
best_acc = grid.score(X_test, y_test)
print(f'Best test set accuracy:\n\t {best_acc}\nAchieved with hyperparameters:\n\t {best_hyperparams}')

Best test set accuracy:
	 0.8130081300813008
Achieved with hyperparameters:
	 {'classifier': LogisticRegression(), 'features__pca__n_components': 3, 'features__select_best__k': 1}


In [22]:
# Serilization of my model 
import pickle
pickle.dump( grid, open( "model.pkl", "wb" ) )

In [31]:
y_p = grid.predict(X_test)
y_p

array(['Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y',
       'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y',
       'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y',
       'Y', 'N', 'N', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'N', 'Y', 'Y', 'Y',
       'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y',
       'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y',
       'Y', 'Y', 'N', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y',
       'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y',
       'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'N', 'Y', 'Y',
       'Y', 'Y', 'Y', 'N', 'Y', 'Y'], dtype=object)

In [40]:
evaluation = f1_score(y_test, y_p, average = 'binary', pos_label = 'Y')
acc = grid.score(X_test, y_test)
print(f'F1_score: {evaluation} ')
print(f'Test set accuracy: {acc}')

F1_score: 0.8808290155440415 
Test set accuracy: 0.8130081300813008


In [43]:
# printing Pipeline for presentation slides
from sklearn import set_config
set_config(display='diagram')

grid

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('Processing',
                                        ColumnTransformer(transformers=[('numeric',
                                                                         Pipeline(steps=[('impute_mean',
                                                                                          SimpleImputer())]),
                                                                         ['ApplicantIncome',
                                                                          'CoapplicantIncome',
                                                                          'LoanAmount',
                                                                          'Loan_Amount_Term',
                                                                          'Credit_History']),
                                                                        ('categorical',
                                                                         Pipeline(steps=[('impute_mode',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('one-hot-encode',
                                                                                          OneH...
                                                                          'Self_Employed',
                                                                          'Property_Area'])])),
                                       ('scaling', StandardScaler()),
                                       ('features',
                                        FeatureUnion(transformer_list=[('pca',
                                                                        PCA()),
                                                                       ('select_best',
                                                                        SelectKBest())])),
                                       ('classifier', 'passthrough')]),
             param_grid={'classifier': [RandomForestClassifier(),
                                        LogisticRegression(), SVC()],
                         'features__pca__n_components': [3, 5],
                         'features__select_best__k': [1, 3, 6]},
             scoring='accuracy')